In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable

In [ ]:
# One-hot encoding data
h = [1, 0, 0, 0]
e = [0, 1, 0, 0]
l = [0, 0, 1, 0]
o = [0, 0, 0, 1]

In [2]:
torch.manual_seed(777)  # reproducibility

idx2char = ['h', 'i', 'e', 'l', 'o']

# Teach hihell -> ihello
x_data = [[0, 1, 0, 2, 3, 3]]   # hihell
y_data = [1, 0, 2, 3, 3, 4]    # ihello

In [3]:
# As we have one batch of samples, we will change them to variables only once
inputs = Variable(torch.LongTensor(x_data))
labels = Variable(torch.LongTensor(y_data))

num_classes = 5
input_size = 5
embedding_size = 10  # embedding size
hidden_size = 5  # output from the LSTM. 5 to directly predict one-hot
batch_size = 1   # one sentence
sequence_length = 6  # |ihello| == 6
num_layers = 1  # one-layer rnn

In [10]:
class Model(nn.Module):

    def __init__(self):
        super(Model, self).__init__()
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.RNN(input_size=embedding_size,
                          hidden_size=5, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Initialize hidden and cell states
        # (num_layers * num_directions, batch, hidden_size)
        h_0 = Variable(torch.zeros(
            num_layers, x.size(0), hidden_size))

        emb = self.embedding(x)
        emb = emb.view(batch_size, sequence_length, -1)

        # Propagate embedding through RNN
        # Input: (batch, seq_len, embedding_size)
        # h_0: (num_layers * num_directions, batch, hidden_size)
        out, _ = self.rnn(emb, h_0)
        return self.fc(out.view(-1, num_classes))

In [11]:
model = Model()
print(model)

# Set loss and optimizer function
# CrossEntropyLoss = LogSoftmax + NLLLoss
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

Model(
  (embedding): Embedding(5, 10)
  (rnn): RNN(10, 5, batch_first=True)
  (fc): Linear(in_features=5, out_features=5, bias=True)
)


In [12]:
# Train the model
for epoch in range(100):
    outputs = model(inputs)
    optimizer.zero_grad()
#     let pytorch loop through the inputs    
#     for input, label in zip(inputs, lables):
#         hidden, output = model(input, hidden)
#         val, idx = output.max(1)
#         loss += criterion(outputs, labels)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    _, idx = outputs.max(1)
    idx = idx.data.numpy()
    result_str = [idx2char[c] for c in idx.squeeze()]
    print("epoch: %d, loss: %1.3f" % (epoch + 1, loss.data[0]))
    print("Predicted string: ", ''.join(result_str))

print("Learning finished!")

epoch: 1, loss: 1.554
Predicted string:  hlhlho
epoch: 2, loss: 1.065
Predicted string:  ililll
epoch: 3, loss: 0.874
Predicted string:  ihilll
epoch: 4, loss: 0.739
Predicted string:  ihilll
epoch: 5, loss: 0.658
Predicted string:  ihilll
epoch: 6, loss: 0.592
Predicted string:  ehelll
epoch: 7, loss: 0.542
Predicted string:  ehelll
epoch: 8, loss: 0.497
Predicted string:  ehello
epoch: 9, loss: 0.430
Predicted string:  ihello
epoch: 10, loss: 0.377
Predicted string:  ihello
epoch: 11, loss: 0.323
Predicted string:  ihello
epoch: 12, loss: 0.273
Predicted string:  ihello
epoch: 13, loss: 0.235
Predicted string:  ihello
epoch: 14, loss: 0.195
Predicted string:  ihello
epoch: 15, loss: 0.159
Predicted string:  ihello
epoch: 16, loss: 0.133
Predicted string:  ihello
epoch: 17, loss: 0.114
Predicted string:  ihello
epoch: 18, loss: 0.096
Predicted string:  ihello
epoch: 19, loss: 0.081
Predicted string:  ihello
epoch: 20, loss: 0.069
Predicted string:  ihello
epoch: 21, loss: 0.059
Predic